In [16]:
import pandas as pd
import numpy as np

In [17]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

In [18]:
df = pd.read_csv('liver_cirrhosis.csv')

In [19]:
df.head()

N_Days Status     Drug    Age Sex Ascites Hepatomegaly Spiders Edema  \
0    2221      C  Placebo  18499   F       N            Y       N     N   
1    1230      C  Placebo  19724   M       Y            N       Y     N   
2    4184      C  Placebo  11839   F       N            N       N     N   
3    2090      D  Placebo  16467   F       N            N       N     N   
4    2105      D  Placebo  21699   F       N            Y       N     N   

   Bilirubin  Cholesterol  Albumin  Copper  Alk_Phos    SGOT  Tryglicerides  \
0        0.5        149.0     4.04   227.0     598.0   52.70           57.0   
1        0.5        219.0     3.93    22.0     663.0   45.00           75.0   
2        0.5        320.0     3.54    51.0    1243.0  122.45           80.0   
3        0.7        255.0     3.74    23.0    1024.0   77.50           58.0   
4        1.9        486.0     3.54    74.0    1052.0  108.50          109.0   

   Platelets  Prothrombin  Stage  
0      256.0          9.9      1  
1      220.0         10.8      2  
2      225.0         10.0      2  
3      151.0         10.2      2  
4      151.0         11.5      1

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         25000 non-null  int64  
 1   Status         25000 non-null  object 
 2   Drug           25000 non-null  object 
 3   Age            25000 non-null  int64  
 4   Sex            25000 non-null  object 
 5   Ascites        25000 non-null  object 
 6   Hepatomegaly   25000 non-null  object 
 7   Spiders        25000 non-null  object 
 8   Edema          25000 non-null  object 
 9   Bilirubin      25000 non-null  float64
 10  Cholesterol    25000 non-null  float64
 11  Albumin        25000 non-null  float64
 12  Copper         25000 non-null  float64
 13  Alk_Phos       25000 non-null  float64
 14  SGOT           25000 non-null  float64
 15  Tryglicerides  25000 non-null  float64
 16  Platelets      25000 non-null  float64
 17  Prothrombin    25000 non-null  float64
 18  Stage 

In [21]:
y = df['Stage']
X = df.drop('Stage', axis = 1)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size = 0.3, random_state = 0)

In [23]:
cat_cols = make_column_selector(dtype_include=object)(X_train)
cat_cols

['Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

In [24]:
num_cols = make_column_selector(dtype_include=np.number)(X_train)
num_cols

['N_Days',
 'Age',
 'Bilirubin',
 'Cholesterol',
 'Albumin',
 'Copper',
 'Alk_Phos',
 'SGOT',
 'Tryglicerides',
 'Platelets',
 'Prothrombin']

# Step 1 : Preprocessing of numeric and categorical data

In [25]:
data_preprocessed = ColumnTransformer(transformers =[ 
    ('ohe', OneHotEncoder(sparse_output=False, drop ='first'), cat_cols), 
    ('num', MinMaxScaler(), num_cols), ], remainder ='passthrough')

In [26]:
data_preprocessed

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['Status', 'Drug', 'Sex', 'Ascites',
                                  'Hepatomegaly', 'Spiders', 'Edema']),
                                ('num', MinMaxScaler(),
                                 ['N_Days', 'Age', 'Bilirubin', 'Cholesterol',
                                  'Albumin', 'Copper', 'Alk_Phos', 'SGOT',
                                  'Tryglicerides', 'Platelets',
                                  'Prothrombin'])])

# Step 2: Running ML model on preprocessed data

In [27]:
pipe = Pipeline(steps =[ 
    ('data_preprocessed', data_preprocessed), 
    ('model', RandomForestClassifier(n_estimators = 200)), 
    ])

In [28]:
 # prefix the parameter name with the object name followed by __ eg: model__criterion
parameters = {
  'model__n_estimators':[5,10,50, 100],
  'model__min_samples_split': [2,3,4],
  'model__criterion': ['gini', 'entropy'] }

In [30]:
grid_pipeline = GridSearchCV(pipe, parameters)

grid_pipeline.fit(X_train,y_train)
grid_pipeline.best_params_

{'model__criterion': 'entropy',
 'model__min_samples_split': 2,
 'model__n_estimators': 100}

In [31]:
grid_pipeline

GridSearchCV(estimator=Pipeline(steps=[('data_preprocessed',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['Status',
                                                                          'Drug',
                                                                          'Sex',
                                                                          'Ascites',
                                                                          'Hepatomegaly',
                                                                          'Spiders',
                                                                          'Edema']),
                                                                        ('num',
                                                                         MinMaxScaler(),
                                                                         ['N_Days',
                                                                          'Age',
                                                                          'Bilirubin',
                                                                          'Cholesterol',
                                                                          'Albumin',
                                                                          'Copper',
                                                                          'Alk_Phos',
                                                                          'SGOT',
                                                                          'Tryglicerides',
                                                                          'Platelets',
                                                                          'Prothrombin'])])),
                                       ('model',
                                        RandomForestClassifier(n_estimators=200))]),
             param_grid={'model__criterion': ['gini', 'entropy'],
                         'model__min_samples_split': [2, 3, 4],
                         'model__n_estimators': [5, 10, 50, 100]})

In [32]:
y_pred = grid_pipeline.predict(X_test)

In [33]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9510666666666666


# Advantage of using Pipeline

## - Streamlined Workflow: 
Allow to streamline steps such as feature scaling, feature selection, and model fitting into a single pipeline, making it easier to manage and reproduce your experiments.
## - Code Simplicity:
Encapsulate all the preprocessing steps and the model fitting process into a single object. This makes your code cleaner and modular.
## - Prevents Data Leakage: 
It ensure that any data transformations are applied only to the training data and not to the test data. 
## - Parameter Grid Search: 
Works seamlessly with for hyperparameter tuning techniques like grid search or randomized search. 
## - Code Reusability: 
Once you have defined a pipeline for a particular machine learning task, you can easily reuse it with different datasets
